In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 20)




## Raw Data Into DataFrames

In [2]:
# Define file paths - adjust this path to match your project location
# Replace this with the actual path to your work_supa project
project_path = Path(r"C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa")

parquet_files = {
    'universe': project_path / 'universe' / 'universe.parquet',
    'portfolio': project_path / 'portfolio' / 'portfolio.parquet',
    'combined_runs': project_path / 'runs' / 'combined_runs.parquet',
    'run_monitor': project_path / 'runs' / 'run_monitor.parquet',
    'bond_z': project_path / 'historical g spread' / 'bond_z.parquet'
}

print("📊 Pipeline Parquet Files to DataFrames")
print("=" * 50)
print("🔍 Checking parquet file availability:")
for name, path in parquet_files.items():
    status = "✅" if path.exists() else "❌"
    print(f"{status} {name}: {path}")
print()

# Load all parquet files into DataFrames
print("📥 Loading parquet files into DataFrames...")
print("=" * 50)

# Dictionary to store all DataFrames
dataframes = {}

for name, file_path in parquet_files.items():
    if file_path.exists():
        try:
            # Load the parquet file
            df = pd.read_parquet(file_path)
            
            # Store in dictionary
            dataframes[name] = df
            
            # Also create a variable with the same name as the file
            globals()[name] = df
            
            print(f"✅ {name}: {df.shape[0]:,} rows × {df.shape[1]} columns ({file_path.stat().st_size / 1024 / 1024:.1f} MB)")
            
        except Exception as e:
            print(f"❌ Error loading {name}: {e}")
    else:
        print(f"❌ File not found: {file_path}")

print(f"\n📊 Total DataFrames loaded: {len(dataframes)}")

# Simple .info() display for all DataFrames
print("📊 DATAFRAME INFO SUMMARY")
print("=" * 80)


# Simple .info() for all DataFrames
print("📊 DATAFRAME INFO")
print("=" * 80)

for name, df in dataframes.items():
    print(f"\n {name.upper()}")
    print("=" * 40)
    df.info()
    print()

📊 Pipeline Parquet Files to DataFrames
🔍 Checking parquet file availability:
✅ universe: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\universe\universe.parquet
✅ portfolio: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\portfolio\portfolio.parquet
✅ combined_runs: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\combined_runs.parquet
✅ run_monitor: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\runs\run_monitor.parquet
✅ bond_z: C:\Users\Eddy\YTM Capital Dropbox\Eddy Winiarz\Trading\COF\Models\Unfinished Models\Eddy\Python Projects\work_supa\historical g spread\bond_z.parquet

📥 Loading parquet files into DataFrames...
✅ universe: 38,917 rows × 47 columns (4.9 MB)
✅ portfolio: 9,390 rows × 51 columns (1.2 MB)
✅ co

## Creating The z_all_enhanced DataFrame

In [3]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION
print("Creating z_all_enhanced DataFrame (OPTIMIZED)...")
print("=" * 60)

import time
start_time = time.time()

# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

print(f"⚡ Starting with {len(z_all_enhanced):,} rows from BOND_Z")

# OPTIMIZATION 1: Pre-filter UNIVERSE and RUN_MONITOR to only needed columns and CUSIPs
print("\nPre-filtering data for faster joins...")

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

print(f"✅ UNIVERSE filtered: {len(universe_filtered):,} rows (from {len(dataframes['universe']):,})")
print(f"✅ RUN_MONITOR filtered: {len(run_monitor_filtered):,} rows (from {len(dataframes['run_monitor']):,})")
print(f"✅ PORTFOLIO filtered: {len(portfolio_filtered):,} rows (from {len(dataframes['portfolio']):,})")

# OPTIMIZATION 2: Create lookup dictionaries for faster joins
print("\n🔧 Creating lookup dictionaries...")

# UNIVERSE lookup dictionaries
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

# RUN_MONITOR lookup dictionaries
run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

print(f"✅ Lookup dictionaries created for {len(bond_cusips):,} unique CUSIPs")

# OPTIMIZATION 3: Vectorized operations for all joins
print("\n⚡ Performing vectorized joins...")

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Performance summary
end_time = time.time()
execution_time = end_time - start_time

print("\n" + "=" * 60)
print("🎉 z_all_enhanced DataFrame created successfully!")
print(f"⚡ Execution time: {execution_time:.3f} seconds")
print(f"📊 Final shape: {z_all_enhanced.shape[0]:,} rows × {z_all_enhanced.shape[1]} columns")

# Quick validation
print("\nVALIDATION SUMMARY:")
print("=" * 30)
print(f"✅ CUSIP_1 UNIVERSE matches: {z_all_enhanced['Ticker_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 UNIVERSE matches: {z_all_enhanced['Ticker_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_1 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_1'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ CUSIP_2 RUN_MONITOR matches: {z_all_enhanced['Bid/Offer_2'].notna().sum():,}/{len(z_all_enhanced):,}")
print(f"✅ Own_CUSIP_1 positions: {z_all_enhanced['Own_CUSIP_1'].sum():,}")
print(f"✅ Own_CUSIP_2 positions: {z_all_enhanced['Own_CUSIP_2'].sum():,}")

print(f"\n🚀 Performance: {len(z_all_enhanced):,} records processed in {execution_time:.3f}s")
print(f"⚡ Speed: {len(z_all_enhanced)/execution_time:,.0f} records/second")

Creating z_all_enhanced DataFrame (OPTIMIZED)...
Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00
⚡ Starting with 1,906,128 rows from BOND_Z

Pre-filtering data for faster joins...
✅ UNIVERSE filtered: 1,813 rows (from 38,917)
✅ RUN_MONITOR filtered: 5,031 rows (from 5,830)
✅ PORTFOLIO filtered: 654 rows (from 9,390)

🔧 Creating lookup dictionaries...
✅ Lookup dictionaries created for 1,953 unique CUSIPs

⚡ Performing vectorized joins...

🎉 z_all_enhanced DataFrame created successfully!
⚡ Execution time: 3.135 seconds
📊 Final shape: 1,906,128 rows × 47 columns

VALIDATION SUMMARY:
✅ CUSIP_1 UNIVERSE matches: 1,752,997/1,906,128
✅ CUSIP_2 UNIVERSE matches: 1,785,979/1,906,128
✅ CUSIP_1 RUN_MONITOR matches: 862,764/1,906,128
✅ CUSIP_2 RUN_MONITOR matches: 847,188/1,906,128
✅ Own_CUSIP_1 positions: 122,990
✅ Own_CUSIP_2 positions: 99,538

🚀 Performance: 1,906,128 records processed in 3.135s
⚡ Speed: 607,990 records/second


## Edits To z_all_enhanced

In [4]:
# Create z_all_enhanced DataFrame - OPTIMIZED VERSION (CLEAN)
# Start with all columns from BOND_Z
z_all_enhanced = dataframes['bond_z'].copy()

# Get the last date from universe and portfolio
last_universe_date = dataframes['universe']['Date'].max()
last_portfolio_date = dataframes['portfolio']['Date'].max()

print(f"Last universe date: {last_universe_date}")
print(f"Last portfolio date: {last_portfolio_date}")

# Define UNIVERSE columns to join
universe_columns = ['Custom_Sector',
    'CPN TYPE', 'Ticker', 'G Sprd', 'Yrs (Cvn)', 'Currency', 
    'Equity Ticker', 'Yrs Since Issue', 'Risk', 'Rating', 
    'Yrs Since Issue Bucket', 'Yrs (Mat) Bucket'
]

# Define RUN_MONITOR columns to join
run_monitor_columns = [
    'Bid/Offer', 'Dealer @ Best Bid', 'Dealer @ Best Offer', 
    'Size @ Best Bid', 'Size @ Best Offer'
]

# Get unique CUSIPs from BOND_Z
bond_cusips = set(z_all_enhanced['CUSIP_1'].unique()) | set(z_all_enhanced['CUSIP_2'].unique())

# Filter UNIVERSE to only needed CUSIPs and columns, and only last date
universe_filtered = dataframes['universe'][
    (dataframes['universe']['CUSIP'].isin(bond_cusips)) &
    (dataframes['universe']['Date'] == last_universe_date)
][['CUSIP'] + universe_columns].copy()

# Filter RUN_MONITOR to only needed CUSIPs and columns  
run_monitor_filtered = dataframes['run_monitor'][
    dataframes['run_monitor']['CUSIP'].isin(bond_cusips)
][['CUSIP'] + run_monitor_columns].copy()

# Filter PORTFOLIO to only last date
portfolio_filtered = dataframes['portfolio'][
    dataframes['portfolio']['Date'] == last_portfolio_date
].copy()

# Create lookup dictionaries for faster joins
universe_lookup_1 = {}
universe_lookup_2 = {}
for _, row in universe_filtered.iterrows():
    cusip = row['CUSIP']
    universe_lookup_1[cusip] = {f"{col}_1": row[col] for col in universe_columns}
    universe_lookup_2[cusip] = {f"{col}_2": row[col] for col in universe_columns}

run_monitor_lookup_1 = {}
run_monitor_lookup_2 = {}
for _, row in run_monitor_filtered.iterrows():
    cusip = row['CUSIP']
    run_monitor_lookup_1[cusip] = {f"{col}_1": row[col] for col in run_monitor_columns}
    run_monitor_lookup_2[cusip] = {f"{col}_2": row[col] for col in run_monitor_columns}

# PORTFOLIO CUSIP set for fast membership testing (only last date)
portfolio_cusips = set(portfolio_filtered['CUSIP'].unique())

# Add UNIVERSE data for CUSIP_1
for col in universe_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in universe_lookup_1.items()}
    )

# Add UNIVERSE data for CUSIP_2  
for col in universe_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in universe_lookup_2.items()}
    )

# Add RUN_MONITOR data for CUSIP_1
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_1"] = z_all_enhanced['CUSIP_1'].map(
        {k: v[f"{col}_1"] for k, v in run_monitor_lookup_1.items()}
    )

# Add RUN_MONITOR data for CUSIP_2
for col in run_monitor_columns:
    z_all_enhanced[f"{col}_2"] = z_all_enhanced['CUSIP_2'].map(
        {k: v[f"{col}_2"] for k, v in run_monitor_lookup_2.items()}
    )

# Add ownership columns (vectorized) - only for last portfolio date
z_all_enhanced['Own_CUSIP_1'] = z_all_enhanced['CUSIP_1'].isin(portfolio_cusips).astype(int)
z_all_enhanced['Own_CUSIP_2'] = z_all_enhanced['CUSIP_2'].isin(portfolio_cusips).astype(int)

# Delete specified columns
columns_to_delete = [
    'Dealer @ Best Bid_1',
    'Size @ Best Bid_1', 
    'Dealer @ Best Offer_2',
    'Size @ Best Offer_2'
]
z_all_enhanced = z_all_enhanced.drop(columns=columns_to_delete)

# Add CR01 columns
risk_1_numeric = pd.to_numeric(z_all_enhanced['Risk_1'], errors='coerce')
risk_2_numeric = pd.to_numeric(z_all_enhanced['Risk_2'], errors='coerce')

z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] = (
    (z_all_enhanced['Size @ Best Offer_1'].fillna(0) / 1000000) * risk_1_numeric.fillna(0)
)

z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] = (
    (z_all_enhanced['Size @ Best Bid_2'].fillna(0) / 1000000) * risk_2_numeric.fillna(0)
)

# Move CUSIP_1 and CUSIP_2 to the end
cols_without_cusips = [col for col in z_all_enhanced.columns if col not in ['CUSIP_1', 'CUSIP_2']]
z_all_enhanced = z_all_enhanced[cols_without_cusips + ['CUSIP_1', 'CUSIP_2']]

# Sort by Z_Score high to low (descending)
z_all_enhanced = z_all_enhanced.sort_values('Z_Score', ascending=False).reset_index(drop=True)

# Define numeric columns to round to 1 decimal (excluding CR01 and Size columns)
numeric_cols_1_decimal = [
    'Last_Spread', 'Z_Score', 'Max', 'Min', 'Last_vs_Max', 'Last_vs_Min', 'Percentile',
    'G Sprd_1', 'Yrs (Cvn)_1', 'Yrs Since Issue_1', 'G Sprd_2', 'Yrs (Cvn)_2', 'Yrs Since Issue_2',
    'Bid/Offer_1', 'Bid/Offer_2'
]

# Apply formatting within display() only
def format_for_display(df):
    """Format DataFrame for display without changing underlying data"""
    # Set display options to show full content
    pd.set_option('display.max_colwidth', None)  # Show full column content
    pd.set_option('display.width', None)  # Don't wrap wide tables
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', None)  # Show all rows
    
    formatted_df = df.copy()
    
    # Round numeric columns to 1 decimal
    for col in numeric_cols_1_decimal:
        if col in formatted_df.columns:
            formatted_df[col] = formatted_df[col].round(1)
    
    # Format CR01 columns with thousand separators and no decimals
    formatted_df['CUSIP_1 CR01 @ Wide Offer'] = formatted_df['CUSIP_1 CR01 @ Wide Offer'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['CUSIP_2 CR01 @ Best Bid'] = formatted_df['CUSIP_2 CR01 @ Best Bid'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Size columns with thousand separators and no decimals
    formatted_df['Size @ Best Offer_1'] = formatted_df['Size @ Best Offer_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Size @ Best Bid_2'] = formatted_df['Size @ Best Bid_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    # Format Risk columns with thousand separators and no decimals
    formatted_df['Risk_1'] = formatted_df['Risk_1'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    formatted_df['Risk_2'] = formatted_df['Risk_2'].apply(
        lambda x: f"{x:,.0f}" if pd.notna(x) else ""
    )
    
    return formatted_df

display(format_for_display(z_all_enhanced.head(20)))

Last universe date: 2025-07-16 00:00:00
Last portfolio date: 2025-06-30 00:00:00


,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
0,BMO 4.609 09/10/25,EAGCCT 1.273 07/17/25,287.5,9.0,289.0,-20.8,-1.4,308.3,99.4,Dep Note,FIXED,BMO,13.1,0.2,CAD,BMO CN Equity,14.8,16,AA-,>7,0-0.25,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,2.0,BMO,"2,000,000",NaN,NBF,"10,000,000",0,0,32,6,063679EC4,26948ZAY8
1,BMO 3.19 03/01/28,EAGCCT 1.273 07/17/25,316.7,9.0,319.7,11.1,-3.0,305.6,99.4,Dep Note,FIXED,BMO,43.8,2.6,CAD,BMO CN Equity,7.4,250,AA,>7,2.1-3.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,4.0,SCM,"10,000,000",NaN,NBF,"10,000,000",0,0,"2,502",6,06368BTX6,26948ZAY8
2,BNS 3.1 02/02/28,EAGCCT 1.273 07/17/25,313.3,9.0,315.9,8.8,-2.6,304.5,99.4,Dep Note,FIXED,BNS,40.0,2.6,CAD,BNS CN Equity,7.4,243,AA-,>7,2.1-3.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,3.0,BMO,"5,000,000",NaN,NBF,"10,000,000",0,0,"1,214",6,064151S95,26948ZAY8
3,ALCTRA 4.309 10/30/34,EAGCCT 1.273 07/17/25,353.0,9.0,354.5,48.8,-1.5,304.2,99.4,Utility,FIXED,ALCTRA,78.6,9.0,CAD,1487407D CN Equity,0.7,746,A-,0.50-1,7.1-10.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,4.0,TD,"2,000,000",NaN,NBF,"10,000,000",0,0,"1,491",6,014443AQ8,26948ZAY8
4,BCECN 3.6 09/29/27,EAGCCT 1.273 07/17/25,336.8,9.0,339.8,33.0,-3.0,303.9,99.4,Cable/Telco,FIXED,BCECN,63.9,2.0,CAD,BCE CN Equity,7.8,188,BBB,>7,2.1-3.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,5.0,NBF,"5,000,000",NaN,NBF,"10,000,000",0,0,942,6,07813ZBY9,26948ZAY8
5,DIS 3.057 03/30/27,EAGCCT 1.273 07/17/25,321.0,9.0,323.6,17.0,-2.7,304.0,99.4,Non Financial Maple,FIXED,DIS,47.7,1.7,CAD,DIS US Equity,5.3,165,A-,5-7,1-2.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,4.0,CIBC,"5,000,000",NaN,NBF,"10,000,000",0,0,824,6,254687FU5,26948ZAY8
6,CAT 4.51 03/05/27,EAGCCT 1.273 07/17/25,318.0,9.0,319.9,12.0,-2.0,306.0,99.4,Industrials,FIXED,CAT,44.0,1.6,CAD,CAT US Equity,1.4,161,A,1-2,1-2.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,5.0,TD,"3,000,000",NaN,NBF,"10,000,000",0,0,483,6,14913LAC4,26948ZAY8
7,ALCTRA 4.627 06/13/34,EAGCCT 1.273 07/17/25,355.8,9.0,358.4,52.0,-2.6,303.9,99.4,Utility,FIXED,ALCTRA,82.5,8.7,CAD,1487407D CN Equity,1.1,730,A-,1-2,7.1-10.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,5.0,RBC,"3,000,000",NaN,NBF,"10,000,000",0,0,"2,189",6,014443AN5,26948ZAY8
8,CTLONE 5.417 09/29/25,EAGCCT 1.273 07/17/25,321.3,9.0,323.9,12.9,-2.6,308.4,99.4,Non Major Senior Unsecured,FIXED,CTLONE,48.0,0.2,CAD,1568Z CN Equity,2.8,21,NR,2-3,0-0.25,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,5.0,CIBC,"15,000,000",NaN,NBF,"10,000,000",0,0,317,6,154728AU3,26948ZAY8
9,CM 4.9 04/02/27,EAGCCT 1.273 07/17/25,320.9,9.0,323.7,13.5,-2.7,307.4,99.4,Bail In,VARIABLE,CM,47.8,0.7,CAD,CM CN Equity,1.3,71,A,1-2,1-2.1,Asset Backed,FIXED,EAGCCT,-275.9,0.0,CAD,751152Z CN Equity,4.9,1,AAA,3-5,0-0.25,4.0,CIBC,"20,000,000",NaN,NBF,"10,000,000",0,0,"1,422",6,13607L3Y7,26948ZAY8


In [5]:
z_all_enhanced = z_all_enhanced[
    (z_all_enhanced['Yrs (Cvn)_1'] > 1) & 
    (z_all_enhanced['Yrs (Cvn)_2'] > 1)
].copy()

In [6]:
# All CAD Bonds IN the Universe

# Create z_cad_universe DataFrame - only CAD currency pairs
z_cad_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Yrs (Cvn)_1'] > 1) & 
    (z_all_enhanced['Yrs (Cvn)_2'] > 1)
].copy()

display(format_for_display(z_cad_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2165,GZMCN 4.83 06/02/53,LOWMAT 2.433 05/14/31,73.2,7.1,73.2,49.6,0.0,23.6,99.8,Utility,FIXED,GZMCN,108.5,27.9,CAD,1118Z CN Equity,2.1,"1,490",A,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,TD,"5,000,000",10.0,TD,"3,000,000",0,0,"7,448","1,503",29280XAD9,548243AJ3
2167,HYDONE 4 12/22/51,LOWMAT 2.433 05/14/31,70.6,7.1,70.6,45.5,0.0,25.1,99.8,Utility,FIXED,HYDONE,106.5,26.4,CAD,H CN Equity,13.6,"1,334",A,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,SCM,"3,000,000",10.0,TD,"3,000,000",0,0,"4,003","1,503",44810ZBE9,548243AJ3
2175,HYDONE 3.79 07/31/62,LOWMAT 2.433 05/14/31,79.8,6.9,79.8,53.8,0.0,26.0,99.8,Utility,FIXED,HYDONE,115.4,37.0,CAD,H CN Equity,13.0,"1,445",A,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,RBC,"5,000,000",10.0,TD,"3,000,000",0,0,"7,226","1,503",44810ZBG4,548243AJ3
2179,ALCTRA 5.225 11/14/52,LOWMAT 2.433 05/14/31,69.4,6.8,69.4,46.5,0.0,22.9,99.8,Utility,FIXED,ALCTRA,104.5,26.8,CAD,1487407D CN Equity,2.7,"1,539",A-,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,1.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"3,077","1,503",014443AM7,548243AJ3
2188,GZMCN 4.67 09/27/32,LOWMAT 2.433 05/14/31,42.0,6.6,42.0,15.2,0.0,26.7,99.8,Utility,FIXED,GZMCN,77.7,7.0,CAD,1118Z CN Equity,2.8,613,A,2-3,7.1-10.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,1.0,TD,"3,000,000",10.0,TD,"3,000,000",0,0,"1,840","1,503",29280XAC1,548243AJ3
2194,HYDONE 4.46 01/27/53,LOWMAT 2.433 05/14/31,70.9,6.5,70.9,45.6,0.0,25.3,99.8,Utility,FIXED,HYDONE,105.6,27.5,CAD,H CN Equity,2.5,"1,430",A,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,TD,"5,000,000",10.0,TD,"3,000,000",0,0,"7,151","1,503",44810ZCM0,548243AJ3
2195,FTSCN 4.862 05/26/53,LOWMAT 2.433 05/14/31,73.6,6.5,73.6,48.4,0.0,25.2,99.8,Utility,FIXED,FTSCN,109.7,27.9,CAD,FTS CN Equity,2.1,"1,493",A-,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"2,986","1,503",34957ZBA1,548243AJ3
2197,EPCOR 3.287 06/28/51,LOWMAT 2.433 05/14/31,72.5,6.5,72.5,46.4,0.0,26.0,99.8,Utility,FIXED,EPCOR,108.2,26.0,CAD,1431Z CN Equity,4.0,"1,209",A-,3-5,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,TD,"2,000,000",10.0,TD,"3,000,000",0,0,"2,418","1,503",29410ZAS4,548243AJ3
2198,FTSCN 3.98 10/23/52,LOWMAT 2.433 05/14/31,72.4,6.5,72.4,47.6,0.0,24.8,99.8,Utility,FIXED,FTSCN,108.3,27.3,CAD,FTS CN Equity,12.7,"1,340",A-,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,NaN,None,,10.0,TD,"3,000,000",0,0,0,"1,503",34957ZAQ7,548243AJ3
2208,HYDONE 3.64 04/05/50,LOWMAT 2.433 05/14/31,71.6,6.5,71.6,45.8,0.0,25.8,99.8,Utility,FIXED,HYDONE,107.0,24.7,CAD,H CN Equity,6.3,"1,246",A,5-7,10.1-25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,SCM,"2,000,000",10.0,TD,"3,000,000",0,0,"2,492","1,503",44810ZCA6,548243AJ3


In [7]:
# Comparing Same Term

z_term = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2'])
].copy()

display(format_for_display(z_term.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2331,FTSCN 2.42 07/18/31,LOWMAT 2.433 05/14/31,19.9,5.7,19.9,-10.7,0.0,30.6,99.8,Utility,FIXED,FTSCN,56.1,6.0,CAD,FTS CN Equity,4.2,514,A-,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,10.0,TD,"2,000,000",10.0,TD,"3,000,000",0,0,"1,028","1,503",34959ZAL6,548243AJ3
2335,EPCOR 2.411 06/30/31,LOWMAT 2.433 05/14/31,19.8,5.6,19.8,-4.6,0.0,24.5,99.8,Utility,FIXED,EPCOR,56.0,6.0,CAD,1431Z CN Equity,4.0,507,A-,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,NaN,None,,10.0,TD,"3,000,000",0,0,0,"1,503",29410ZAU9,548243AJ3
2349,HTHROW 3.661 01/13/31,LOWMAT 2.433 05/14/31,61.5,5.5,61.5,31.9,0.0,29.6,99.8,Non Financial Maple,FIXED,HTHROW,97.3,5.5,CAD,8226864Z LN Equity,4.8,481,BBB+,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,RBC,"2,000,000",10.0,TD,"3,000,000",0,0,963,"1,503",42246QAF5,548243AJ3
2378,GZMCN 3.04 02/09/32,LOWMAT 2.433 05/14/31,37.6,5.4,37.6,11.7,0.0,25.9,99.8,Utility,FIXED,GZMCN,73.5,6.6,CAD,1118Z CN Equity,3.4,555,A,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,4.0,TD,"2,000,000",10.0,TD,"3,000,000",0,0,"1,110","1,503",29280XAA5,548243AJ3
2398,HTHROW 3.782 09/04/30,LOWMAT 2.433 05/14/31,57.9,5.2,57.9,30.1,0.0,27.8,99.8,Non Financial Maple,FIXED,HTHROW,93.8,5.1,CAD,8226864Z LN Equity,6.9,457,BBB+,5-7,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,2.0,RBC,"2,000,000",10.0,TD,"3,000,000",0,0,914,"1,503",42246QAD0,548243AJ3
2402,GLCRTR 4.74 09/20/26,HOMEQU 7.108 12/11/26,-89.6,5.1,-89.6,-136.9,0.0,47.4,99.8,Asset Backed,FIXED,GLCRTR,49.2,1.2,CAD,CTC/A CN Equity,1.1,117,AAA,1-2,1-2.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,CIBC,"2,000,000",NaN,CIBC,"3,000,000",1,1,233,418,37638ZBW3,43739CAA9
2403,DE 4.95 06/14/27,HOMEQU 7.108 12/11/26,-98.8,5.1,-96.8,-140.2,-2.0,41.4,99.0,Industrials,FIXED,DE,44.5,1.9,CAD,DE US Equity,2.1,188,A,2-3,1-2.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,942,418,47788ZAN1,43739CAA9
2440,BCECN 3 03/17/31,LOWMAT 2.433 05/14/31,56.3,4.9,56.3,31.0,0.0,25.3,99.8,Cable/Telco,FIXED,BCECN,92.6,5.7,CAD,BCE CN Equity,4.3,487,BBB,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,4.0,RBC,"5,000,000",10.0,TD,"3,000,000",0,0,"2,433","1,503",07813ZCG7,548243AJ3
2443,HYDONE 2.23 09/17/31,LOWMAT 2.433 05/14/31,1.1,4.9,1.1,-18.2,0.0,19.3,99.8,Utility,FIXED,HYDONE,38.4,6.2,CAD,H CN Equity,3.8,527,A,3-5,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,NaN,None,,10.0,TD,"3,000,000",0,0,0,"1,503",44810ZCG3,548243AJ3
2489,ENBGAS 4.15 08/17/32,LOWMAT 2.433 05/14/31,39.3,4.6,39.3,11.6,0.0,27.8,99.8,Utility,FIXED,ENBGAS,75.3,6.8,CAD,ENB CN Equity,2.9,595,A-,2-3,5.1-7.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,RBC,"2,000,000",10.0,TD,"3,000,000",0,0,"1,189","1,503",29260ZAG1,548243AJ3


In [8]:
# Comparing Same Sector

z_sector = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2'])
].copy()

display(format_for_display(z_sector.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2165,GZMCN 4.83 06/02/53,LOWMAT 2.433 05/14/31,73.2,7.1,73.2,49.6,0.0,23.6,99.8,Utility,FIXED,GZMCN,108.5,27.9,CAD,1118Z CN Equity,2.1,"1,490",A,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,TD,"5,000,000",10.0,TD,"3,000,000",0,0,"7,448","1,503",29280XAD9,548243AJ3
2167,HYDONE 4 12/22/51,LOWMAT 2.433 05/14/31,70.6,7.1,70.6,45.5,0.0,25.1,99.8,Utility,FIXED,HYDONE,106.5,26.4,CAD,H CN Equity,13.6,"1,334",A,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,SCM,"3,000,000",10.0,TD,"3,000,000",0,0,"4,003","1,503",44810ZBE9,548243AJ3
2175,HYDONE 3.79 07/31/62,LOWMAT 2.433 05/14/31,79.8,6.9,79.8,53.8,0.0,26.0,99.8,Utility,FIXED,HYDONE,115.4,37.0,CAD,H CN Equity,13.0,"1,445",A,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,RBC,"5,000,000",10.0,TD,"3,000,000",0,0,"7,226","1,503",44810ZBG4,548243AJ3
2179,ALCTRA 5.225 11/14/52,LOWMAT 2.433 05/14/31,69.4,6.8,69.4,46.5,0.0,22.9,99.8,Utility,FIXED,ALCTRA,104.5,26.8,CAD,1487407D CN Equity,2.7,"1,539",A-,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,1.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"3,077","1,503",014443AM7,548243AJ3
2188,GZMCN 4.67 09/27/32,LOWMAT 2.433 05/14/31,42.0,6.6,42.0,15.2,0.0,26.7,99.8,Utility,FIXED,GZMCN,77.7,7.0,CAD,1118Z CN Equity,2.8,613,A,2-3,7.1-10.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,1.0,TD,"3,000,000",10.0,TD,"3,000,000",0,0,"1,840","1,503",29280XAC1,548243AJ3
2194,HYDONE 4.46 01/27/53,LOWMAT 2.433 05/14/31,70.9,6.5,70.9,45.6,0.0,25.3,99.8,Utility,FIXED,HYDONE,105.6,27.5,CAD,H CN Equity,2.5,"1,430",A,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,TD,"5,000,000",10.0,TD,"3,000,000",0,0,"7,151","1,503",44810ZCM0,548243AJ3
2195,FTSCN 4.862 05/26/53,LOWMAT 2.433 05/14/31,73.6,6.5,73.6,48.4,0.0,25.2,99.8,Utility,FIXED,FTSCN,109.7,27.9,CAD,FTS CN Equity,2.1,"1,493",A-,2-3,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,3.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"2,986","1,503",34957ZBA1,548243AJ3
2197,EPCOR 3.287 06/28/51,LOWMAT 2.433 05/14/31,72.5,6.5,72.5,46.4,0.0,26.0,99.8,Utility,FIXED,EPCOR,108.2,26.0,CAD,1431Z CN Equity,4.0,"1,209",A-,3-5,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,TD,"2,000,000",10.0,TD,"3,000,000",0,0,"2,418","1,503",29410ZAS4,548243AJ3
2198,FTSCN 3.98 10/23/52,LOWMAT 2.433 05/14/31,72.4,6.5,72.4,47.6,0.0,24.8,99.8,Utility,FIXED,FTSCN,108.3,27.3,CAD,FTS CN Equity,12.7,"1,340",A-,>7,>25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,NaN,None,,10.0,TD,"3,000,000",0,0,0,"1,503",34957ZAQ7,548243AJ3
2208,HYDONE 3.64 04/05/50,LOWMAT 2.433 05/14/31,71.6,6.5,71.6,45.8,0.0,25.8,99.8,Utility,FIXED,HYDONE,107.0,24.7,CAD,H CN Equity,6.3,"1,246",A,5-7,10.1-25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,SCM,"2,000,000",10.0,TD,"3,000,000",0,0,"2,492","1,503",44810ZCA6,548243AJ3


In [9]:
# Comparing Same Ticker

z_ticker = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_ticker.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2702,RCICN 3.65 03/31/27,RCICN 5.9 09/21/33,-51.4,4.2,-51.4,-67.2,0.0,15.8,99.8,Cable/Telco,FIXED,RCICN,70.6,1.5,CAD,RCI/B CN Equity,5.3,150,BBB-,5-7,1-2.1,Cable/Telco,FIXED,RCICN,123.4,7.9,CAD,RCI/B CN Equity,1.8,692,BBB-,1-2,7.1-10.1,4.0,NBF,"10,000,000",4.0,RBC,"10,000,000",0,0,"1,499","6,921",775109BK6,775109DA6
3713,RCICN 3.65 03/31/27,RCICN 5.8 09/21/30,-24.7,3.6,-24.7,-41.2,0.0,16.5,99.8,Cable/Telco,FIXED,RCICN,70.6,1.5,CAD,RCI/B CN Equity,5.3,150,BBB-,5-7,1-2.1,Cable/Telco,FIXED,RCICN,95.9,5.0,CAD,RCI/B CN Equity,1.8,471,BBB-,1-2,5.1-7.1,4.0,NBF,"10,000,000",5.0,BMO,"5,000,000",0,0,"1,499","2,355",775109BK6,775109CW9
4180,LOWMAT 2.307 10/21/26,LOWMAT 2.433 05/14/31,-5.7,3.5,-5.7,-35.0,0.0,29.2,99.8,Utility,FIXED,LOWMAT,31.5,1.3,CAD,1644Z CN Equity,8.8,123,A+,>7,1-2.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,5.0,TD,"2,000,000",10.0,TD,"3,000,000",0,0,246,"1,503",548243AH7,548243AJ3
5011,RCICN 3.65 03/31/27,RCICN 4 ¼ 04/15/32,-38.8,3.4,-38.2,-57.0,-0.6,18.1,98.6,Cable/Telco,FIXED,RCICN,70.6,1.5,CAD,RCI/B CN Equity,5.3,150,BBB-,5-7,1-2.1,Cable/Telco,FIXED,RCICN,110.5,6.8,CAD,RCI/B CN Equity,3.3,578,BBB-,3-5,5.1-7.1,4.0,NBF,"10,000,000",4.0,RBC,"10,000,000",0,0,"1,499","5,776",775109BK6,775109BV2
5408,RY 4.642 01/17/28,RY 5.228 06/24/30,-10.5,3.3,-9.6,-20.2,-0.9,9.8,99.4,Bail In,FIXED,RY,52.3,2.5,CAD,RY CN Equity,2.5,243,A+,2-3,2.1-3.1,Bail In,FIXED,RY,61.9,4.9,CAD,RY CN Equity,2.1,466,A+,2-3,4.1-5.1,3.0,BMO,"5,000,000",5.0,TD,"10,000,000",1,0,"1,216","4,657",780086WG5,780086ZH0
5465,FTSCN 2.54 07/13/50,FTSCN 3.12 05/11/50,-4.9,3.3,-4.4,-17.6,-0.5,12.7,98.2,Utility,FIXED,FTSCN,109.8,25.0,CAD,FTS CN Equity,5.0,"1,086",A-,5-7,10.1-25.1,Utility,FIXED,FTSCN,115.9,24.8,CAD,FTS CN Equity,5.2,"1,155",BBB+,5-7,10.1-25.1,5.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,172","5,773",34959ZAK8,34958ZAE3
5603,RY 4.632 05/01/28,RY 5.228 06/24/30,-5.3,3.3,-4.7,-20.5,-0.7,15.1,99.4,Bail In,FIXED,RY,57.3,2.8,CAD,RY CN Equity,2.2,270,A+,2-3,2.1-3.1,Bail In,FIXED,RY,61.9,4.9,CAD,RY CN Equity,2.1,466,A+,2-3,4.1-5.1,3.0,TD,"10,000,000",5.0,TD,"10,000,000",1,0,"2,700","4,657",780086XL3,780086ZH0
5863,RY 4.612 07/26/27,RY 5.228 06/24/30,-11.9,3.3,-10.5,-23.8,-1.4,12.0,97.4,Bail In,FIXED,RY,51.4,2.0,CAD,RY CN Equity,3.0,198,A+,2-3,1-2.1,Bail In,FIXED,RY,61.9,4.9,CAD,RY CN Equity,2.1,466,A+,2-3,4.1-5.1,3.0,BMO,"5,000,000",5.0,TD,"10,000,000",0,0,991,"4,657",780086VK7,780086ZH0
6504,FTSCN 2.82 08/09/49,FTSCN 3.12 05/11/50,-4.0,3.2,-3.5,-16.6,-0.5,12.6,97.8,Utility,FIXED,FTSCN,111.2,24.1,CAD,FTS CN Equity,5.9,"1,113",A-,5-7,10.1-25.1,Utility,FIXED,FTSCN,115.9,24.8,CAD,FTS CN Equity,5.2,"1,155",BBB+,5-7,10.1-25.1,5.0,TD,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,226","5,773",34959ZAJ1,34958ZAE3
6759,BNS 3.807 11/15/28,BNS 4.68 02/01/29,-0.6,3.2,-0.1,-12.1,-0.5,11.6,99.0,Bail In,VARIABLE,BNS,62.9,2.3,CAD,BNS CN Equity,0.7,224,A,0.50-1,3.1-4.1,Bail In,FIXED,BNS,63.9,3.6,CAD,BNS CN Equity,1.4,338,A,1-2,3.1-4.1,3.0,CIBC,"5,000,000",3.0,BMO,"20,000,000",0,1,"1,120","6,756",06418MX74,064164QM1


In [10]:
# Where CAD Bonds May Appear Cheap vs USD Bonds


z_cad_vs_usd = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'USD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_cad_vs_usd.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
142528,ENBCN 5 ⅜ 09/27/2077,ENBCN 7 ⅜ 01/15/2083,19.8,1.8,26.7,-128.3,-6.8,148.1,98.6,Non Financial Hybrids,VARIABLE,ENBCN,208.2,2.2,CAD,ENB CN Equity,7.8,208,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,212.1,2.3,USD,ENB CN Equity,2.8,213,BB+,2-3,>25.1,10.0,BMO,"5,000,000",NaN,NaN,,0,0,"1,040",0,29250NAT2,29250NBN4
161711,ENBCN 6 ⅝ 04/12/2078,ENBCN 7 ⅜ 01/15/2083,11.9,1.7,20.1,-151.0,-8.2,162.9,98.2,Non Financial Hybrids,VARIABLE,ENBCN,198.6,2.7,CAD,ENB CN Equity,7.3,261,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,212.1,2.3,USD,ENB CN Equity,2.8,213,BB+,2-3,>25.1,10.0,RBC,"2,000,000",NaN,NaN,,0,0,523,0,29250NAX3,29250NBN4
226695,ENBCN 5 01/19/2082,ENBCN 7 ⅜ 01/15/2083,44.9,1.5,67.7,-94.0,-22.8,138.9,95.8,Non Financial Hybrids,VARIABLE,ENBCN,231.3,6.3,CAD,ENB CN Equity,3.5,516,BB+,3-5,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,212.1,2.3,USD,ENB CN Equity,2.8,213,BB+,2-3,>25.1,8.0,TD,"2,000,000",NaN,NaN,,0,0,"1,032",0,29250NBJ3,29250NBN4
306371,ENBCN 5 ⅜ 09/27/2077,ENBCN 6 ¼ 03/01/2078,-16.8,1.2,-10.1,-107.0,-6.8,90.2,92.3,Non Financial Hybrids,VARIABLE,ENBCN,208.2,2.2,CAD,ENB CN Equity,7.8,208,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,ENBCN,254.0,2.6,USD,ENB CN Equity,7.4,240,BB+,>7,>25.1,10.0,BMO,"5,000,000",NaN,NaN,,0,0,"1,040",0,29250NAT2,29250NAW5
375768,TRPCN 4.65 05/18/2077,TRPCN 5 ½ 09/15/2079,11.0,1.1,27.0,-119.7,-16.0,130.7,92.3,Non Financial Hybrids,VARIABLE,TRPCN,201.6,1.8,CAD,TRP CN Equity,8.2,174,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,TRPCN,211.4,4.2,USD,TRP CN Equity,5.8,366,BB+,5-7,>25.1,20.0,TD,"3,000,000",NaN,NaN,,0,0,523,0,89356BAD0,89356BAE8
393309,F 5.242 05/23/28,F 6.8 05/12/28,40.0,1.0,54.1,-54.8,-14.1,94.8,85.1,Auto,FIXED,F,211.9,2.8,CAD,F US Equity,1.1,258,BB+,1-2,2.1-3.1,Auto,FIXED,F,197.3,2.7,USD,F US Equity,2.3,256,BB+,2-3,2.1-3.1,6.0,RBC,"3,000,000",NaN,NaN,,0,0,773,0,34527ACY4,345397C92
403067,F 5.581 02/22/27,F 5.8 03/05/27,38.7,1.0,62.0,-37.3,-23.3,76.1,83.1,Auto,FIXED,F,178.7,1.6,CAD,F US Equity,1.4,155,BB+,1-2,1-2.1,Auto,FIXED,F,161.8,1.6,USD,F US Equity,1.5,148,BB+,1-2,1-2.1,3.0,SCM,"5,000,000",NaN,NaN,,0,0,775,0,34527ACW8,345397D83
460870,TRPCN 4.2 03/04/2081,TRPCN 5 ½ 09/15/2079,27.2,0.9,46.4,-54.2,-19.2,81.4,85.5,Non Financial Hybrids,VARIABLE,TRPCN,216.3,5.4,CAD,TRP CN Equity,4.4,450,BB+,3-5,>25.1,Non Financial Hybrids,VARIABLE,TRPCN,211.4,4.2,USD,TRP CN Equity,5.8,366,BB+,5-7,>25.1,5.0,RBC,"2,000,000",NaN,NaN,,0,0,900,0,89356BAF5,89356BAE8
468455,F 5.581 02/22/27,F 5.85 05/17/27,30.4,0.9,44.5,-34.9,-14.1,65.3,82.7,Auto,FIXED,F,178.7,1.6,CAD,F US Equity,1.4,155,BB+,1-2,1-2.1,Auto,FIXED,F,174.0,1.8,USD,F US Equity,1.2,166,BB+,1-2,1-2.1,3.0,SCM,"5,000,000",NaN,NaN,,0,0,775,0,34527ACW8,345397E74
505668,TRPCN 4.65 05/18/2077,TRPCN 5.6 03/07/2082,2.4,0.8,26.0,-121.9,-23.6,124.3,77.2,Non Financial Hybrids,VARIABLE,TRPCN,201.6,1.8,CAD,TRP CN Equity,8.2,174,BB+,>7,>25.1,Non Financial Hybrids,VARIABLE,TRPCN,242.5,6.4,USD,TRP CN Equity,3.4,516,BB+,3-5,>25.1,20.0,TD,"3,000,000",NaN,NaN,,0,0,523,0,89356BAD0,89356BAG3


In [11]:
# Where CAD Bonds May Appear Rich vs USD Bonds


z_usd_vs_cad = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'USD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Custom_Sector_1'] == z_all_enhanced['Custom_Sector_2']) &
    (z_all_enhanced['Yrs (Mat) Bucket_1'] == z_all_enhanced['Yrs (Mat) Bucket_2']) &
    (z_all_enhanced['Ticker_1'] == z_all_enhanced['Ticker_2'])
].copy()

display(format_for_display(z_usd_vs_cad.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
34027,ALACN 7.2 10/15/54,ALACN 8.9 11/10/2083,66.3,2.4,66.3,-35.4,0.0,101.7,99.8,Non Financial Hybrids,VARIABLE,ALACN,332.1,9.0,USD,ALA CN Equity,0.8,660,BB,0.50-1,>25.1,Non Financial Hybrids,VARIABLE,ALACN,212.4,3.3,CAD,ALA CN Equity,1.7,324,BB,1-2,>25.1,NaN,NaN,,16.0,CIBC,"2,000,000",0,0,0,649,021361AD2,021361AC4
53998,DE 4 ¾ 01/20/28,DE 4.38 07/11/28,-20.8,2.2,-18.7,-49.8,-2.2,29.0,96.6,Industrials,FIXED,DE,65.3,2.5,USD,DE US Equity,2.5,238,A,2-3,2.1-3.1,Industrials,FIXED,DE,55.8,3.0,CAD,DE US Equity,1.0,287,A,1-2,2.1-3.1,NaN,NaN,,4.0,RBC,"5,000,000",0,0,0,"1,434",24422EWR6,47788ZAS0
55288,ALACN 7.2 10/15/54,ALACN 7.35 08/17/2082,86.9,2.2,87.9,-17.6,-1.1,104.5,99.4,Non Financial Hybrids,VARIABLE,ALACN,332.1,9.0,USD,ALA CN Equity,0.8,660,BB,0.50-1,>25.1,Non Financial Hybrids,VARIABLE,ALACN,192.6,2.1,CAD,ALA CN Equity,2.9,204,BB,2-3,>25.1,NaN,NaN,,24.0,CIBC,"2,000,000",0,0,0,408,021361AD2,021361AB6
73344,RCICN 3.8 03/15/32,RCICN 4 ¼ 04/15/32,54.1,2.1,57.2,14.7,-3.0,39.4,97.8,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,Cable/Telco,FIXED,RCICN,110.5,6.8,CAD,RCI/B CN Equity,3.3,578,BBB-,3-5,5.1-7.1,NaN,NaN,,4.0,RBC,"10,000,000",0,0,0,"5,776",775109CC3,775109BV2
76882,PLD 1 ¼ 10/15/30,PLD 5 ¼ 01/15/31,-33.2,2.1,-32.8,-73.2,-0.3,40.1,98.2,REIT,FIXED,PLD,92.6,5.3,USD,PLD US Equity,4.9,422,A,3-5,5.1-7.1,REIT,FIXED,PLD,84.6,5.3,CAD,PLD US Equity,2.7,486,A,2-3,5.1-7.1,NaN,NaN,,5.0,NBF,"5,000,000",0,1,0,"2,431",74340XBR1,74340XCD1
82999,RCICN 3.8 03/15/32,RCICN 5.8 09/21/30,68.2,2.0,70.2,29.6,-2.0,38.6,97.4,Cable/Telco,FIXED,RCICN,148.6,6.7,USD,RCI/B CN Equity,3.3,534,BBB-,3-5,5.1-7.1,Cable/Telco,FIXED,RCICN,95.9,5.0,CAD,RCI/B CN Equity,1.8,471,BBB-,1-2,5.1-7.1,NaN,NaN,,5.0,BMO,"5,000,000",0,0,0,"2,355",775109CC3,775109CW9
86070,TOYOTA 4 ⅝ 01/12/28,TOYOTA 4.33 01/24/28,-11.9,2.0,-10.3,-38.4,-1.6,26.5,99.4,Auto,FIXED,TOYOTA,67.9,2.5,USD,7203 JP Equity,2.5,235,A+,2-3,2.1-3.1,Auto,FIXED,TOYOTA,52.3,2.5,CAD,7203 JP Equity,2.5,244,A+,2-3,2.1-3.1,NaN,NaN,,5.0,NBF,"5,000,000",0,0,0,"1,219",89236TKQ7,892329BQ5
91810,PLD 1 ⅝ 03/15/31,PLD 5 ¼ 01/15/31,-16.6,2.0,-15.9,-56.0,-0.7,39.4,99.4,REIT,FIXED,PLD,110.7,5.7,USD,PLD US Equity,4.4,449,A,3-5,5.1-7.1,REIT,FIXED,PLD,84.6,5.3,CAD,PLD US Equity,2.7,486,A,2-3,5.1-7.1,NaN,NaN,,5.0,NBF,"5,000,000",0,1,0,"2,431",74340XBS9,74340XCD1
95564,PLD 2 ¼ 01/15/32,PLD 5 ¼ 01/15/31,-15.3,2.0,-15.3,-56.5,0.0,41.2,99.8,REIT,FIXED,PLD,114.3,6.5,USD,PLD US Equity,2.8,507,A,2-3,5.1-7.1,REIT,FIXED,PLD,84.6,5.3,CAD,PLD US Equity,2.7,486,A,2-3,5.1-7.1,NaN,NaN,,5.0,NBF,"5,000,000",0,1,0,"2,431",74340XCB5,74340XCD1
99407,BCECN 3.65 08/15/52,BCECN 5.6 08/11/53,-38.5,2.0,-37.8,-87.4,-0.7,48.8,99.4,Cable/Telco,FIXED,BCECN,201.7,27.1,USD,BCE CN Equity,3.4,995,BBB,3-5,>25.1,Cable/Telco,FIXED,BCECN,154.7,27.6,CAD,BCE CN Equity,1.9,"1,458",BBB,1-2,>25.1,NaN,NaN,,4.0,TD,"5,000,000",0,0,0,"7,289",0778FPAJ8,07813ZCP7


In [12]:
# Which Bonds in Our Portfolio May Look Rich vs Universe

z_port_vs_universe = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2373,HICITR 4.065 02/22/59,HOMEQU 7.108 12/11/26,57.5,5.4,58.0,22.0,-0.5,35.5,99.4,Other,FIXED,HICITR,200.8,20.6,CAD,1690248D CN Equity,6.4,"1,056",NR,5-7,>25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,418,42834JAB8,43739CAA9
2375,HICITR 3.91 02/22/49,HOMEQU 7.108 12/11/26,79.8,5.4,79.8,42.7,0.0,37.1,99.8,Other,FIXED,HICITR,222.5,13.7,CAD,1690248D CN Equity,6.4,841,NR,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,NaN,NaN,,NaN,CIBC,"3,000,000",0,1,0,418,42834JAA0,43739CAA9
2402,GLCRTR 4.74 09/20/26,HOMEQU 7.108 12/11/26,-89.6,5.1,-89.6,-136.9,0.0,47.4,99.8,Asset Backed,FIXED,GLCRTR,49.2,1.2,CAD,CTC/A CN Equity,1.1,117,AAA,1-2,1-2.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,CIBC,"2,000,000",NaN,CIBC,"3,000,000",1,1,233,418,37638ZBW3,43739CAA9
2403,DE 4.95 06/14/27,HOMEQU 7.108 12/11/26,-98.8,5.1,-96.8,-140.2,-2.0,41.4,99.0,Industrials,FIXED,DE,44.5,1.9,CAD,DE US Equity,2.1,188,A,2-3,1-2.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,942,418,47788ZAN1,43739CAA9
2408,DE 5.17 09/15/28,HOMEQU 7.108 12/11/26,-86.3,5.1,-85.7,-124.3,-0.6,38.0,99.0,Industrials,FIXED,DE,57.1,3.2,CAD,DE US Equity,1.8,307,A,1-2,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,4.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,536",418,47788ZAP6,43739CAA9
2412,DE 4.38 07/11/28,HOMEQU 7.108 12/11/26,-87.6,5.1,-86.9,-125.1,-0.7,37.5,99.0,Industrials,FIXED,DE,55.8,3.0,CAD,DE US Equity,1.0,287,A,1-2,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,4.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,434",418,47788ZAS0,43739CAA9
2426,DE 4.63 04/04/29,HOMEQU 7.108 12/11/26,-82.5,5.0,-81.5,-120.5,-1.0,38.0,99.0,Industrials,FIXED,DE,60.7,3.7,CAD,DE US Equity,1.3,354,A,1-2,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"1,768",418,47788ZAR2,43739CAA9
2442,DE 1.34 09/08/27,HOMEQU 7.108 12/11/26,-122.1,4.9,-121.1,-158.8,-1.0,36.6,99.0,Industrials,FIXED,DE,20.8,2.1,CAD,DE US Equity,4.8,203,A,3-5,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,NaN,None,,NaN,CIBC,"3,000,000",0,1,0,418,47788ZAH4,43739CAA9
2445,CPXCN 5.816 09/15/28,HOMEQU 7.108 12/11/26,-50.5,4.9,-50.5,-87.4,-0.0,36.8,99.4,Power Gen/Renewable,FIXED,CPXCN,92.3,3.1,CAD,CPX CN Equity,1.8,299,BBB-,1-2,3.1-4.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,7.0,SCM,"3,000,000",NaN,CIBC,"3,000,000",0,1,897,418,14046ZAP4,43739CAA9
2451,BRUPOW 4.7 06/21/31,HOMEQU 7.108 12/11/26,-52.7,4.8,-51.4,-88.7,-1.3,36.0,99.0,Power Gen/Renewable,FIXED,BRUPOW,90.1,5.8,CAD,561679Z CN Equity,1.3,519,BBB,1-2,5.1-7.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,SCM,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,595",418,116705AN4,43739CAA9


In [13]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute

z_port_vs_universe_2way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
3732,ALTALK 4.872 11/15/40,WSPCN 4.754 09/12/34,4.9,3.6,4.9,-18.2,0.0,23.1,99.8,Utility,FIXED,ALTALK,121.4,15.3,CAD,BRK/A US Equity,14.7,"1,067",A-,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,2.0,NBF,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,134","3,625",02135ZAD8,92938WAD5
4059,CUCN 3.805 09/10/42,WSPCN 4.754 09/12/34,1.7,3.5,1.7,-19.7,0.0,21.4,99.8,Utility,FIXED,CUCN,119.0,17.2,CAD,ACO/X CN Equity,12.8,"1,056",A,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,NBF,"3,000,000",NaN,NBF,"5,000,000",0,1,"3,169","3,625",12657ZAY9,92938WAD5
4146,MRUCN 4.657 02/07/33,WSPCN 4.754 09/12/34,-30.1,3.5,-28.8,-41.2,-1.3,11.1,99.4,Retail/Grocers,FIXED,MRUCN,88.7,7.3,CAD,MRU CN Equity,2.4,634,BBB,2-3,7.1-10.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,TD,"5,000,000",NaN,NBF,"5,000,000",0,1,"3,172","3,625",59162NAL3,92938WAD5
4156,CUCN 4.543 10/24/41,WSPCN 4.754 09/12/34,2.6,3.5,2.6,-17.7,0.0,20.4,99.8,Utility,FIXED,CUCN,120.6,16.3,CAD,ACO/X CN Equity,13.7,"1,084",A,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,NBF,"3,000,000",NaN,NBF,"5,000,000",0,1,"3,251","3,625",12657ZAW3,92938WAD5
4192,BRUPOW 4 06/21/30,WSPCN 4.754 09/12/34,-36.8,3.5,-36.8,-52.4,0.0,15.6,99.8,Power Gen/Renewable,FIXED,BRUPOW,82.0,4.7,CAD,561679Z CN Equity,5.3,426,BBB,5-7,4.1-5.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,TD,"5,000,000",NaN,NBF,"5,000,000",0,1,"2,130","3,625",116705AJ3,92938WAD5
4218,ALTALK 4.462 11/08/41,WSPCN 4.754 09/12/34,2.6,3.5,2.6,-19.5,0.0,22.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,1.0,NBF,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,159","3,625",02135ZAE6,92938WAD5
4511,MRUCN 3.39 12/06/27,WSPCN 4.754 09/12/34,-67.5,3.5,-67.5,-84.2,0.0,16.7,99.8,Retail/Grocers,FIXED,MRUCN,51.1,2.1,CAD,MRU CN Equity,7.6,206,BBB,>7,2.1-3.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,RBC,"10,000,000",NaN,NBF,"5,000,000",0,1,"2,059","3,625",59162NAF6,92938WAD5
4619,ENBGAS 5.2 07/23/40,WSPCN 4.754 09/12/34,12.0,3.4,12.0,-8.7,-0.0,20.8,99.4,Utility,FIXED,ENBGAS,130.9,15.0,CAD,ENB CN Equity,15.0,"1,063",A-,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,4.0,BMO,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,126","3,625",90664ZAT7,92938WAD5
5135,GTAAIR 5.63 06/07/40,WSPCN 4.754 09/12/34,-2.5,3.4,-2.5,-24.8,0.0,22.3,99.8,Airport,FIXED,GTAAIR,116.4,14.9,CAD,1318Z CN Equity,15.1,"1,109",A+,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,NBF,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,218","3,625",39191ZAW9,92938WAD5
5195,FTSCN 5.677 11/08/33,WSPCN 4.754 09/12/34,-9.1,3.4,-8.7,-27.9,-0.4,18.8,99.4,Utility,FIXED,FTSCN,110.2,8.1,CAD,FTS CN Equity,1.7,702,BBB,1-2,7.1-10.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,5.0,NBF,"5,000,000",NaN,NBF,"5,000,000",0,1,"3,510","3,625",349553AR8,92938WAD5


In [14]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least 1 Side

z_port_vs_universe_1way_liquidity = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquidity.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2451,BRUPOW 4.7 06/21/31,HOMEQU 7.108 12/11/26,-52.7,4.8,-51.4,-88.7,-1.3,36.0,99.0,Power Gen/Renewable,FIXED,BRUPOW,90.1,5.8,CAD,561679Z CN Equity,1.3,519,BBB,1-2,5.1-7.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,SCM,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,595",418,116705AN4,43739CAA9
2504,CPXCN 4.831 09/16/31,HOMEQU 7.108 12/11/26,-13.3,4.6,-11.8,-47.2,-1.5,33.9,99.0,Power Gen/Renewable,FIXED,CPXCN,130.2,6.0,CAD,CPX CN Equity,0.8,527,BBB-,0.50-1,5.1-7.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,4.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,634",418,14046ZAS8,43739CAA9
2520,CPXCN 5.973 01/25/34,HOMEQU 7.108 12/11/26,8.1,4.5,8.5,-27.7,-0.4,35.8,99.4,Power Gen/Renewable,FIXED,CPXCN,150.8,8.3,CAD,CPX CN Equity,1.6,702,BBB-,1-2,7.1-10.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,7.0,SCM,"3,000,000",NaN,CIBC,"3,000,000",0,1,"2,107",418,14046ZAR0,43739CAA9
2531,BRUPOW 4 06/21/30,HOMEQU 7.108 12/11/26,-61.2,4.5,-59.9,-99.3,-1.3,38.1,99.0,Power Gen/Renewable,FIXED,BRUPOW,82.0,4.7,CAD,561679Z CN Equity,5.3,426,BBB,5-7,4.1-5.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,130",418,116705AJ3,43739CAA9
2586,BNCN 5.95 06/14/35,HOMEQU 7.108 12/11/26,-8.3,4.4,-8.0,-53.7,-0.3,45.4,99.4,Asset Manager,FIXED,BNCN,134.1,9.9,CAD,BN CN Equity,20.1,820,A-,>7,7.1-10.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,10.0,TD,"5,000,000",NaN,CIBC,"3,000,000",0,1,"4,099",418,10549PAJ0,43739CAA9
2599,GTAAIR 4.53 12/02/41,HOMEQU 7.108 12/11/26,-30.4,4.4,-28.9,-71.7,-1.6,41.2,99.0,Airport,FIXED,GTAAIR,113.0,16.4,CAD,1318Z CN Equity,13.6,"1,098",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,294",418,39191ZAY5,43739CAA9
2610,GTAAIR 5.3 02/25/41,HOMEQU 7.108 12/11/26,-29.0,4.3,-27.4,-71.5,-1.6,42.5,99.0,Airport,FIXED,GTAAIR,114.3,15.6,CAD,1318Z CN Equity,14.4,"1,122",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,367",418,39191ZAX7,43739CAA9
2631,BMO 3.19 03/01/28,HOMEQU 7.108 12/11/26,-99.7,4.3,-98.7,-142.3,-1.1,42.6,99.0,Dep Note,FIXED,BMO,43.8,2.6,CAD,BMO CN Equity,7.4,250,AA,>7,2.1-3.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,4.0,SCM,"10,000,000",NaN,CIBC,"3,000,000",0,1,"2,502",418,06368BTX6,43739CAA9
2655,GTAAIR 5.63 06/07/40,HOMEQU 7.108 12/11/26,-26.9,4.3,-25.2,-71.4,-1.6,44.5,99.0,Airport,FIXED,GTAAIR,116.4,14.9,CAD,1318Z CN Equity,15.1,"1,109",A+,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,5.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,218",418,39191ZAW9,43739CAA9
2656,BIP 5.71 07/27/30,HOMEQU 7.108 12/11/26,-44.8,4.3,-44.7,-82.1,-0.1,37.3,99.4,Infastructure,FIXED,BIP,97.0,4.9,CAD,BIP US Equity,2.0,456,BBB+,1-2,4.1-5.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,6.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",1,1,"2,282",418,11291ZAM9,43739CAA9


In [15]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute Both Sides At Reasonable Bid/Offer

z_port_vs_universe_2way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['CUSIP_2 CR01 @ Best Bid'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Bid/Offer_2'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_2way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2


In [16]:
# Which Bonds in Our Portfolio May Look Rich vs Universe & We Can Execute At Least One Side At Reasonable Bid/Offer

z_port_vs_universe_1way_liquiditybidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <2) &
    (z_all_enhanced['Own_CUSIP_2'] == 1)
].copy()

display(format_for_display(z_port_vs_universe_1way_liquiditybidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2728,ALTALK 4.054 11/21/44,HOMEQU 7.108 12/11/26,-26.6,4.1,-26.6,-66.3,0.0,39.6,99.8,Utility,FIXED,ALTALK,114.7,19.4,CAD,BRK/A US Equity,10.6,"1,156",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,1.0,NBF,"3,000,000",NaN,CIBC,"3,000,000",0,1,"3,468",418,02135ZAP1,43739CAA9
2746,ALTALK 4.462 11/08/41,HOMEQU 7.108 12/11/26,-21.8,4.1,-21.8,-66.1,0.0,44.3,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,1.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,159",418,02135ZAE6,43739CAA9
2877,ALTALK 4.09 06/30/45,HOMEQU 7.108 12/11/26,-28.7,4.0,-27.9,-67.7,-0.8,38.9,99.0,Utility,FIXED,ALTALK,114.4,20.0,CAD,BRK/A US Equity,10.0,"1,179",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,1.0,NBF,"5,000,000",NaN,CIBC,"3,000,000",0,1,"5,894",418,02135ZAQ9,43739CAA9
2934,ENBGAS 4.88 06/21/41,HOMEQU 7.108 12/11/26,-15.0,3.9,-13.5,-53.2,-1.5,38.1,99.0,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,0.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,165",418,90664ZAU4,43739CAA9
4218,ALTALK 4.462 11/08/41,WSPCN 4.754 09/12/34,2.6,3.5,2.6,-19.5,0.0,22.1,99.8,Utility,FIXED,ALTALK,119.3,16.3,CAD,BRK/A US Equity,13.7,"1,080",A-,>7,10.1-25.1,Industrials,FIXED,WSPCN,118.9,8.9,CAD,WSP CN Equity,0.8,725,NR,0.50-1,7.1-10.1,1.0,NBF,"2,000,000",NaN,NBF,"5,000,000",0,1,"2,159","3,625",02135ZAE6,92938WAD5
4376,ALTALK 5.381 03/26/40,HOMEQU 7.108 12/11/26,-20.5,3.5,-19.4,-64.2,-1.1,43.8,99.4,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,143",418,02135ZAC0,43739CAA9
4530,CUCN 4.851 06/03/52,HOMEQU 7.108 12/11/26,-19.1,3.5,-17.3,-54.8,-1.7,35.8,99.4,Utility,FIXED,CUCN,122.7,26.9,CAD,ACO/X CN Equity,3.1,"1,430",A-,3-5,>25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,860",418,136717BD6,43739CAA9
5700,BIP 5.95 07/27/53,HOMEQU 7.108 12/11/26,20.8,3.3,24.2,-10.1,-3.4,30.9,99.0,Infastructure,FIXED,BIP,165.4,27.5,CAD,BIP US Equity,2.0,"1,472",BBB+,1-2,>25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,0.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,945",418,11291ZAN7,43739CAA9
5844,FTSCN 3.85 12/07/48,HOMEQU 7.108 12/11/26,-31.0,3.3,-30.1,-71.1,-0.9,40.1,99.0,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,471",418,34959ZAH5,43739CAA9
8110,ALACN 2.477 11/30/30,HOMEQU 7.108 12/11/26,-52.2,3.1,-52.2,-92.1,0.0,39.9,99.8,Midstream,FIXED,ALACN,87.7,5.4,CAD,ALA CN Equity,4.6,459,BBB-,3-5,5.1-7.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,-2.0,BMO,"5,000,000",NaN,CIBC,"3,000,000",0,1,"2,293",418,02138ZAX1,43739CAA9


In [17]:
# Bonds That May Look Cheap and We Can Execute At Least One Side At Reasonable Bid/Offer

z_cad_universe_cr01_bidoffer = z_all_enhanced[
    (z_all_enhanced['Currency_1'] == 'CAD') & 
    (z_all_enhanced['Currency_2'] == 'CAD') &
    (z_all_enhanced['CUSIP_1 CR01 @ Wide Offer'] >2000) &
    (z_all_enhanced['Bid/Offer_1'] <1) 
].copy()

display(format_for_display(z_cad_universe_cr01_bidoffer.head(20)))

,Security_1,Security_2,Last_Spread,Z_Score,Max,Min,Last_vs_Max,Last_vs_Min,Percentile,Custom_Sector_1,CPN TYPE_1,Ticker_1,G Sprd_1,Yrs (Cvn)_1,Currency_1,Equity Ticker_1,Yrs Since Issue_1,Risk_1,Rating_1,Yrs Since Issue Bucket_1,Yrs (Mat) Bucket_1,Custom_Sector_2,CPN TYPE_2,Ticker_2,G Sprd_2,Yrs (Cvn)_2,Currency_2,Equity Ticker_2,Yrs Since Issue_2,Risk_2,Rating_2,Yrs Since Issue Bucket_2,Yrs (Mat) Bucket_2,Bid/Offer_1,Dealer @ Best Offer_1,Size @ Best Offer_1,Bid/Offer_2,Dealer @ Best Bid_2,Size @ Best Bid_2,Own_CUSIP_1,Own_CUSIP_2,CUSIP_1 CR01 @ Wide Offer,CUSIP_2 CR01 @ Best Bid,CUSIP_1,CUSIP_2
2242,FTSCN 3.85 12/07/48,LOWMAT 2.433 05/14/31,76.4,6.2,76.4,49.4,0.0,27.0,99.8,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,-1.0,BMO,"2,000,000",10.0,TD,"3,000,000",0,0,"2,471","1,503",34959ZAH5,548243AJ3
2934,ENBGAS 4.88 06/21/41,HOMEQU 7.108 12/11/26,-15.0,3.9,-13.5,-53.2,-1.5,38.1,99.0,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,0.0,NBF,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,165",418,90664ZAU4,43739CAA9
3186,ENBGAS 4.88 06/21/41,LOWMAT 2.433 05/14/31,92.4,3.8,92.4,61.4,0.0,30.9,99.8,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,0.0,NBF,"2,000,000",10.0,TD,"3,000,000",0,0,"2,165","1,503",90664ZAU4,548243AJ3
3247,FTSCN 3.85 12/07/48,T 4.85 05/25/47,-42.9,3.8,-42.9,-61.4,0.0,18.5,99.8,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Cable/Telco,FIXED,T,157.8,21.9,CAD,T US Equity,8.1,"1,220",BBB,>7,10.1-25.1,-1.0,BMO,"2,000,000",5.0,RBC,"2,000,000",0,0,"2,471","2,440",34959ZAH5,00206RDY5
3920,BCECN 2 ½ 05/14/30,LOWMAT 2.433 05/14/31,37.5,3.6,37.5,16.2,0.0,21.3,99.8,Cable/Telco,FIXED,BCECN,73.1,4.8,CAD,BCE CN Equity,5.2,423,BBB,5-7,4.1-5.1,Utility,FIXED,LOWMAT,49.4,5.8,CAD,1644Z CN Equity,4.2,501,A+,3-5,5.1-7.1,-4.0,BMO,"5,000,000",10.0,TD,"3,000,000",0,0,"2,117","1,503",07813ZCE2,548243AJ3
4190,BCECN 2 ½ 05/14/30,RCICN 4 ¼ 12/09/49,-64.6,3.5,-64.3,-95.4,-0.3,30.8,99.4,Cable/Telco,FIXED,BCECN,73.1,4.8,CAD,BCE CN Equity,5.2,423,BBB,5-7,4.1-5.1,Cable/Telco,FIXED,RCICN,137.8,24.4,CAD,RCI/B CN Equity,5.6,"1,247",BBB-,5-7,10.1-25.1,-4.0,BMO,"5,000,000",6.0,NBF,"5,000,000",0,0,"2,117","6,235",07813ZCE2,775109CR0
4364,ENBGAS 4.88 06/21/41,RCICN 3.3 12/10/29,50.4,3.5,50.8,24.8,-0.4,25.5,99.4,Utility,FIXED,ENBGAS,128.2,15.9,CAD,ENB CN Equity,14.1,"1,083",A-,>7,10.1-25.1,Cable/Telco,FIXED,RCICN,78.8,4.4,CAD,RCI/B CN Equity,5.6,398,BBB-,5-7,4.1-5.1,0.0,NBF,"2,000,000",5.0,BMO,"5,000,000",0,0,"2,165","1,988",90664ZAU4,775109CQ2
4376,ALTALK 5.381 03/26/40,HOMEQU 7.108 12/11/26,-20.5,3.5,-19.4,-64.2,-1.1,43.8,99.4,Utility,FIXED,ALTALK,122.1,14.7,CAD,BRK/A US Equity,15.3,"1,072",A-,>7,10.1-25.1,Mortgage Provider,FIXED,HOMEQU,142.8,1.4,CAD,HEQ CN Equity,1.6,139,NR,1-2,1-2.1,-1.0,BMO,"2,000,000",NaN,CIBC,"3,000,000",0,1,"2,143",418,02135ZAC0,43739CAA9
5069,FTSCN 3.85 12/07/48,HYDONE 1.69 01/16/31,83.8,3.4,83.8,66.4,0.0,17.4,99.8,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Utility,FIXED,HYDONE,32.4,5.5,CAD,H CN Equity,4.8,472,A,3-5,5.1-7.1,-1.0,BMO,"2,000,000",NaN,RBC,"5,000,000",0,0,"2,471","2,359",34959ZAH5,44810ZCF5
5511,FTSCN 3.85 12/07/48,RCICN 4 ¼ 12/09/49,-25.7,3.3,-25.1,-70.1,-0.6,44.3,98.2,Utility,FIXED,FTSCN,112.0,23.4,CAD,FTS CN Equity,6.6,"1,235",A-,5-7,10.1-25.1,Cable/Telco,FIXED,RCICN,137.8,24.4,CAD,RCI/B CN Equity,5.6,"1,247",BBB-,5-7,10.1-25.1,-1.0,BMO,"2,000,000",6.0,NBF,"5,000,000",0,0,"2,471","6,235",34959ZAH5,775109CR0
